In [26]:
import gensim
import numpy as np
import pandas as pd
import os
import re
import csv
import codecs
import sys
import pickle

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

from keras import callbacks
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import concatenate,Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

import matplotlib as mpl
%matplotlib inline
from matplotlib import pyplot as plt
from keras.utils import plot_model 
from IPython.display import Image
import pydot
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config)


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

from string import punctuation

from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints


In [27]:
# Specify the folder locations
#W2V_DIR = './data/GoogleNews-vectors-negative300.bin' #
GloVe_DIR = './data/glove.twitter.27B.50d.txt'
#the data directory
DATA_DIR = './data'
# These are some hyperparameters that can be tuned
MAX_SENT_LEN = 150 #75(0.68), 150, 300 700(90% but too time comsuming)
MAX_VOCAB_SIZE = 1119000 #vocabulary
LSTM_DIM = 100#len(embd[0])
EMBEDDING_DIM = 50 #50 for GloVe 300 for w2v
BATCH_SIZE = 128
N_EPOCHS = 40 #40


In [28]:
# fix random seed for reproducibility
seed = 1
np.random.seed(seed)

In [29]:
bodies = pd.read_csv(DATA_DIR + '/body_table.csv')

test_df = pd.read_csv(DATA_DIR + '/test_data.csv')

In [30]:
test_df.replace('unrelated',0,True)
test_df.replace('agree',1,True)
test_df.replace('disagree',2,True)
test_df.replace('discuss',3,True)

In [31]:
combine_df_test = test_df.join(bodies.set_index('Body ID'), on='Body ID')

In [32]:
word_seq_head_test = [text_to_word_sequence(head) for head in combine_df_test['Headline']]
word_seq_bodies_test = [text_to_word_sequence(body) for body in combine_df_test['articleBody']]

In [33]:
word_seq = []

for i in range(len(word_seq_head_test)):
    word_seq.append(word_seq_head_test[i])
for i in range(len(word_seq_bodies_test)):
    word_seq.append(word_seq_bodies_test[i])



In [34]:
word_seq_test = [list(i) for i in word_seq_head_test]
for i in range(len(word_seq_head_test)):
    word_seq_test[i].extend(word_seq_bodies_test[i]) 


In [35]:
filter_list = '!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters=filter_list)
tokenizer.fit_on_texts([seq for seq in word_seq])

print("Number of words in vocabulary:", len(tokenizer.word_index))

Number of words in vocabulary: 10213


In [36]:
print('90th Percentile Sentence of headline:', np.percentile([len(seq) for seq in word_seq_head_test], 90))

90th Percentile Sentence of headline: 16.0


In [37]:
print('90th Percentile Sentence of body:', np.percentile([len(seq) for seq in word_seq_bodies_test], 90))

90th Percentile Sentence of body: 585.0


In [38]:
# Convert the sequence of words to sequnce of indices
X_test = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq_test])
X_test = pad_sequences(X_test, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

In [39]:
from keras import models

In [40]:
model_file_path = './result/lstm/epoch40/Bidirectional_lstm_150token_lr0.001_trainable_39_0.9535.h5'
model = models.load_model(model_file_path)

In [41]:
preds = model.predict([X_test])

In [42]:
outputs = [np.argmax(p, axis = -1) for p in preds]

In [43]:
for i in range(len(outputs)):
    if outputs[i] == 0: outputs[i] = "unrelated"
    if outputs[i] == 1: outputs[i] = "disagree"
    if outputs[i] == 2: outputs[i] = "agree"
    if outputs[i] == 3: outputs[i] = "discuss"
        
df_predicted = pd.DataFrame({'Stance': outputs})
result = pd.concat([test_df, df_predicted], axis=1, sort=False)
result.to_csv('./bidrectional_answer.csv', index=False, encoding='utf-8')